# Drishti-AI: Project Overview & Architecture

**Vayu-Mamba (Delhi NCR Edition)** — Hyper-local pollution attribution using satellite, social, and sensor data.

This notebook introduces the project, the four-phase architecture, and the Delhi NCR region with visualizations.

## 1. The Four Phases: Sky, Street, Breath, Mind

| Phase | Modality | Goal |
|-------|----------|------|
| **Sky** | Sentinel-5P (NO2, CO, Aerosol Index) | 64×64 daily pollution "texture" map |
| **Street** | Geo-tagged images/text (CLIP) + Road graph (OSM) | Point cloud + GNN for urban canyons |
| **Breath** | CPCB PM2.5 sensors | Ground truth for training; model predicts when sensors unavailable |
| **Mind** | VLM + Streamlit | Attribution report: "High AQI in Okhla likely due to landfill fire" |

In [ ]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Delhi NCR ROI (from config)
roi = {"min_lon": 76.8, "min_lat": 28.4, "max_lon": 77.5, "max_lat": 28.9}
cities = {
    "Delhi": (77.21, 28.61),
    "Noida": (77.39, 28.54),
    "Gurugram": (77.03, 28.46),
    "Ghaziabad": (77.44, 28.67),
    "Faridabad": (77.32, 28.41),
}

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.set_facecolor("#1a1a2e")
fig.patch.set_facecolor("#1a1a2e")
ax.set_xlim(roi["min_lon"] - 0.1, roi["max_lon"] + 0.1)
ax.set_ylim(roi["min_lat"] - 0.1, roi["max_lat"] + 0.1)
rect = mpatches.Rectangle((roi["min_lon"], roi["min_lat"]), roi["max_lon"] - roi["min_lon"], roi["max_lat"] - roi["min_lat"])
rect.set_facecolor("none")
rect.set_edgecolor("#00d9ff")
rect.set_linewidth(2)
ax.add_patch(rect)
for name, (lon, lat) in cities.items():
    ax.scatter(lon, lat, s=120, c="#e94560", zorder=5, edgecolors="white", linewidths=1.5)
    ax.annotate(name, (lon, lat), xytext=(5, 5), textcoords="offset points", fontsize=10, color="white")
ax.set_xlabel("Longitude", color="#eee")
ax.set_ylabel("Latitude", color="#eee")
ax.tick_params(colors="#aaa")
ax.set_title("Delhi NCR Region of Interest (ROI) — 64×64 grid", color="white", fontsize=14)
plt.tight_layout()
plt.show()

## 2. Data Flow: Offline vs Online Pipelines

```
OFFLINE (batch)                    ONLINE (stream/daily)
Historical GEE + social + CPCB  →  Latest N days → inference input
         ↓                                    ↓
   Training dataset (.pt)              Model prediction (PM2.5 map)
         ↓                                    +
   Full retrain on schedule           Optional: CL update (replay)
```

New sensor data is added when available; model is updated periodically (retrain or incremental CL).

In [ ]:
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.set_xlim(0, 12)
ax.set_ylim(0, 6)
ax.axis("off")
ax.set_facecolor("#0f0f1a")
fig.patch.set_facecolor("#0f0f1a")

def box(ax, x, y, w, h, label, color):
    p = FancyBboxPatch((x, y), w, h, boxstyle="round,pad=0.02", facecolor=color, edgecolor="#444")
    ax.add_patch(p)
    ax.text(x + w/2, y + h/2, label, ha="center", va="center", fontsize=9, color="white", wrap=True)

box(ax, 0.5, 3.5, 2, 1.2, "GEE\nSatellite", "#2d5a27")
box(ax, 0.5, 2.0, 2, 1.2, "Social\n(CLIP + geo)", "#5a2d4a")
box(ax, 0.5, 0.5, 2, 1.2, "CPCB\nPM2.5", "#5a3d2d")
box(ax, 3.5, 2, 2, 2, "Offline\nPipeline", "#1e3a5f")
box(ax, 6.5, 2, 2, 2, "Training\nDataset", "#3d1e5f")
box(ax, 9, 2.5, 2, 1.2, "FusionModel", "#5f1e3d")
box(ax, 9, 0.8, 2, 1.2, "PM2.5 Map", "#4a5f1e")

ax.annotate("", xy=(3.3, 2.5), xytext=(2.6, 2.5), arrowprops=dict(arrowstyle="->", color="#888"))
ax.annotate("", xy=(6.3, 2.5), xytext=(5.6, 2.5), arrowprops=dict(arrowstyle="->", color="#888"))
ax.annotate("", xy=(8.8, 2.5), xytext=(8.5, 2.5), arrowprops=dict(arrowstyle="->", color="#888"))
ax.annotate("", xy=(10, 1.9), xytext=(10, 2.2), arrowprops=dict(arrowstyle="->", color="#888"))
ax.text(5.8, 2.9, "train", fontsize=8, color="#aaa")
ax.text(7.6, 2.9, "predict", fontsize=8, color="#aaa")
ax.set_title("Data flow: Pipelines → Model → Prediction", color="white", fontsize=12)
plt.tight_layout()
plt.show()

## 3. Tech Stack & Repo Structure

- **Pipeline:** `src/pipeline/` — GEE client, grid_align, to_tensor
- **Model:** `src/model/` — VimBaseEncoder, SocialPointEncoder, SparseFusionLayer, FusionModel
- **Scripts:** `scripts/run_offline_pipeline.py`, `run_online_pipeline.py`, `download_ncr_30d.py`
- **Config:** `config/roi.yaml` (ROI + grid 64×64)